In [1]:
import torch as pt
import torchaudio as ta
from torch.utils.data import Dataset,DataLoader

import numpy as numpy

import os
import subprocess
import tqdm as tqdm
import json

import librosa as lb

F:\anacondapython\envs\tf\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [21]:
path='data\public_dataset'
for file in os.listdir(path):
    if file.endswith('webm'):
        name,ext=os.path.splitext(file)
        new_name=name+'.wav'
        os.rename(os.path.join(path,file),os.path.join(path,new_name))

In [ ]:
wav_file=

In [13]:
def convert_webm_to_wav(file):
    command = ['ffmpeg', '-i', file, file[:-5] + '.wav']
    subprocess.run(command)

In [19]:
path='data\public_dataset'
for file in os.listdir(path):
    if file.endswith('webm'):
        subprocess.run(['ffmpeg','-i',os.path.join(path,file),'test_df/'+file[:-5]+'.wav'])

In [15]:
path='data\public_dataset'
name_set=set()
for file in os.listdir(path):
    if file.endswith('wav'):
        name_set.add(file)
print(len(name_set))

0


In [3]:
t=os.path.join(path,list(name_set)[0])

In [9]:
t=t.replace('webm','wav')

In [5]:
print(str(ta.get_audio_backend()))

soundfile


In [8]:
print(t)

data\public_dataset\3432eab1-c4e0-466e-aa62-e839ce0da28e.wav


In [12]:
tt=os.fspath(t)
print(tt)

data\public_dataset\3432eab1-c4e0-466e-aa62-e839ce0da28e.wav


In [4]:
y, sr = lb.load(t)

F:\anacondapython\envs\tf\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


NoBackendError: 

In [18]:
signal,sr=ta.load('AnyConv.com__00a2faca-e1f2-4848-9afe-058f949d3252.wav')

In [13]:
t=t.replace('webm','json')
with open(t,'r') as f:
    content=json.loads(f.read())
print(content)

{'datetime': '2020-04-11T13:35:31.037321+00:00', 'cough_detected': '0.9902', 'latitude': '37.4', 'longitude': '-6.1'}


In [16]:
print(content['cough_detected'])

0.9902


In [ ]:
class CoughVidDataset(Dataset):

    def __init__(self,path):
        name_set=set()
        for file in os.listdir(path):
            if file.endswith('webm'):
                name_set.add(file)
        name_set=list(name_set)
        self.datalist=name_set
        self.path=path
        
    def __len__(self):
        return len(self.datalist)

    def __getitem__(self,idx):
        audio_path=os.path.join(self.path,self.datalist[idx])
